
# Prueba

1. Vamos a cargar el dataset e importamos las librerias. 

2. Tras el análisis del EDA vamos a eliminiar las columnas "author" y "geometry". 

3. A continuación separamos el dataset en dos;
<BR>

i) Incluye los nulos (10.415) que será donde hagamos las prediciones.
<BR>

ii) No incluye los nulos (21.229) y será el que utilicemos para entrenar el modelo.

In [3]:
#Importamos las librerias, cargamos el fichero y vemos las columnas. 

import csv
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('data/data.csv')

data_copia = data.copy()

data_copia.shape


(31644, 10)

In [6]:

# Función para rellenar los valores nulos con la media de la columna dentro de cada grupo de geometría
def fill_nulls_with_group_mean(group):
    return group.fillna(group.mean())

# Rellenar los valores nulos en las columnas seleccionadas con la media de su grupo de geometría
columns_to_fill = ['pressure [MPa]', 'mass_flux [kg/m2-s]', 'D_e [mm]', 'D_h [mm]', 'length [mm]']
df_filled = data_copia.groupby('geometry')[columns_to_fill].transform(fill_nulls_with_group_mean)

# Actualizar las columnas seleccionadas en el DataFrame original
data_copia[columns_to_fill] = df_filled

# Contar los valores nulos que quedan
null_counts = data_copia[columns_to_fill].isnull().sum()

print("Valores nulos después del relleno:")
print(null_counts)





Valores nulos después del relleno:
pressure [MPa]         5500
mass_flux [kg/m2-s]    5500
D_e [mm]               5500
D_h [mm]               5500
length [mm]            5500
dtype: int64


In [7]:
# Los valores nulos que quedan ahora los rellenaoms con la media de su columna. 

# Columnas para rellenar con la media de su columna
columns_to_fill = ['pressure [MPa]', 'mass_flux [kg/m2-s]', 'D_e [mm]', 'D_h [mm]', 'length [mm]']

# Rellenar los valores nulos en las columnas seleccionadas con la media de su columna
data_copia[columns_to_fill] = data[columns_to_fill].fillna(data_copia[columns_to_fill].mean())

# Contar los valores nulos que quedan
null_counts = data_copia[columns_to_fill].isnull().sum()

print("Valores nulos después del relleno:")
print(null_counts)



Valores nulos después del relleno:
pressure [MPa]         0
mass_flux [kg/m2-s]    0
D_e [mm]               0
D_h [mm]               0
length [mm]            0
dtype: int64


In [8]:
# Contar valores nulos en cada columna. 

valores_nulos = data_copia.isnull().sum()

print(valores_nulos)


id                         0
author                  5024
geometry                5500
pressure [MPa]             0
mass_flux [kg/m2-s]        0
x_e_out [-]            10415
D_e [mm]                   0
D_h [mm]                   0
length [mm]                0
chf_exp [MW/m2]            0
dtype: int64


In [9]:
# Separo el nuevo DataFrame en dos:

# Crear un DataFrame con filas que contienen NaN en 'x_e_out [-]'

df_nan = data_copia[data_copia['x_e_out [-]'].isnull()]

# Crear un DataFrame sin filas que contienen NaN en 'x_e_out [-]'

df_sin_nan = data_copia[~data_copia['x_e_out [-]'].isnull()]

print("Data_copia: ", data_copia.shape)
print("Data con los NUN:", df_nan.shape)
print("Data sin los NUN: ", df_sin_nan.shape)


Data_copia:  (31644, 10)
Data con los NUN: (10415, 10)
Data sin los NUN:  (21229, 10)


In [102]:
# Antes de entrenar el modelo creamos el nuevo DF_sin NAN 

new_df_sin_nan = df_sin_nan.drop(['id', 'author', 'geometry'], axis=1)



In [ ]:
# Entrenamos el modelo


# Ahora vamos a entrenar el modelo 

import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


# Separar las características (X) y la variable objetivo (y)
X = new_df_sin_nan.drop('x_e_out [-]', axis=1)
y = df_sin_nan['x_e_out [-]']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un objeto Scaler y ajustarlo a los datos de entrenamiento
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Crear y entrenar el modelo XGBoost con los mejores hiperparámetros
modelo_XGB = XGBRegressor(learning_rate=0.1, max_depth=5, n_estimators=100)
modelo_XGB.fit(X_train_scaled, y_train)


# Realizar predicciones en el conjunto de prueba
y_pred = modelo_XGB.predict(X_test_scaled)

# Calcular el RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)

# Calcular el R cuadrado
r2 = r2_score(y_test, y_pred)

# Calcular el MSE
mse = mean_squared_error(y_test, y_pred)

# Calcular el MAE
mae = mean_absolute_error(y_test, y_pred)

print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared:", r2)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)